In [1]:
!python -V

Python 3.12.4


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

### MLFLOW 

In [6]:
import mlflow
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('my_nyc_taxi_exp1')

<Experiment: artifact_location='file:///D:/cursos/mlops_zoomcamp/homework2/mlruns/1', creation_time=1748100905662, experiment_id='1', last_update_time=1748100905662, lifecycle_stage='active', name='my_nyc_taxi_exp1', tags={}>

In [7]:
def read_dataframe(filename):
    print(f'READING FILE: {filename}')
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
        print(df.columns)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    # print(df.duration.describe())

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # print(df.duration.describe())

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    print('Done')
    
    return df

In [8]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

READING FILE: ./data/yellow_tripdata_2023-01.parquet
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')
Done
READING FILE: ./data/yellow_tripdata_2023-02.parquet
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')
Done


In [9]:
len(df_train), len(df_val)

(3009173, 2855951)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = [ 'PULocationID', 'DOLocationID'] #'PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.811819793542861

In [17]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [18]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

8.138118733385994

## tracking experiments with MLFLOW

In [14]:
with mlflow.start_run():
    mlflow.set_tag('developer','luis')
    mlflow.log_param('train-data-path','./data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param('val-data-path','./data/yellow_tripdata_2023-02.parquet')
    
    alpha=0.001
    mlflow.log_param('alpha',alpha)
    mlflow.log_param('model_type','Lasso')
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse_val = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('RMSE_val',rmse_val)

## Import XGBOOST and HyperOpt

In [15]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:


train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)



In [24]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params = params,
            dtrain = train,
            num_boost_round = 1000,
            evals=[(valid,'validatio')],
            early_stopping_rounds=20,
        )
        y_pred = booster.predict(valid)
        rmse_val = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric('RMSE_val',rmse_val)
    return {'loss':rmse_val, 'status':STATUS_OK}        

In [26]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth',4,100,1)),
    'learning_rate':hp.loguniform('learning_rate',-3,0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42   
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

[0]	validatio-rmse:9.72381                                                                                             
[1]	validatio-rmse:9.44179                                                                                             
[2]	validatio-rmse:9.21088                                                                                             
[3]	validatio-rmse:9.02302                                                                                             
[4]	validatio-rmse:8.86806                                                                                             
[5]	validatio-rmse:8.74259                                                                                             
[6]	validatio-rmse:8.63838                                                                                             
[7]	validatio-rmse:8.55341                                                                                             
[8]	validatio-rmse:8.48284              

KeyboardInterrupt: 

## using The best values 

In [28]:
params={
    'learning_rate':0.205,
    'max_depth':18,
    'min_child_weight':1.24,
    'objective':'reg:squarederror',
    'reg_alpha':0.28,
    'reg_lambda':0.004,
    'seed':42,   
}
mlflow.xgboost.autolog()
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=100,
    evals=[(valid,'validation')],
    early_stopping_rounds=10,
)

2025/05/24 15:21:38 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 3.0.0, but the installed version is 3.0.1. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.
2025/05/24 15:21:39 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c7ae314f8f9a46e3b8913ef66dce940f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2025/05/24 15:23:59 WARNING mlflow.xgboost: Failed to log dataset information to MLflow Tracking. Reason: Unable to allocate 5.77 GiB for an array with shape (3009173, 515) and data type float32


[0]	validation-rmse:9.31803
[1]	validation-rmse:8.80850
[2]	validation-rmse:8.45409
[3]	validation-rmse:8.17959
[4]	validation-rmse:7.99497
[5]	validation-rmse:7.87127
[6]	validation-rmse:7.78404
[7]	validation-rmse:7.62483
[8]	validation-rmse:7.50688
[9]	validation-rmse:7.45840
[10]	validation-rmse:7.41475
[11]	validation-rmse:7.38010
[12]	validation-rmse:7.34836
[13]	validation-rmse:7.31658
[14]	validation-rmse:7.26960
[15]	validation-rmse:7.19015
[16]	validation-rmse:7.17313
[17]	validation-rmse:7.15488
[18]	validation-rmse:7.10211
[19]	validation-rmse:7.08011
[20]	validation-rmse:7.06873
[21]	validation-rmse:7.04177
[22]	validation-rmse:7.02955
[23]	validation-rmse:6.94155
[24]	validation-rmse:6.91620
[25]	validation-rmse:6.89951
[26]	validation-rmse:6.88900
[27]	validation-rmse:6.87571
[28]	validation-rmse:6.86546
[29]	validation-rmse:6.81255
[30]	validation-rmse:6.74572
[31]	validation-rmse:6.73346
[32]	validation-rmse:6.72531
[33]	validation-rmse:6.71731
[34]	validation-rmse:6.7

2025/05/24 15:24:41 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/05/24 15:24:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "D:\cursos\mlops_zoomcamp\homework2\env_hw2\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [15:24:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/05/24 15:25:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Model managment

In [31]:
with mlflow.start_run():
    mlflow.set_tag('developer','luis')
    mlflow.log_param('train-data-path','./data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param('val-data-path','./data/yellow_tripdata_2023-02.parquet')
    
    alpha=0.01
    mlflow.log_param('alpha',alpha)
    mlflow.log_param('model_type','Lasso')
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse_val = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('RMSE_val',rmse_val)

    ### saving model artifact
    mlflow.log_artifact(local_path="models/lin_reg.bin",
                       artifact_path ="models_pickle" 
                      )
    

    

In [32]:
with mlflow.start_run():
    params={
    'learning_rate':0.205,
    'max_depth':18,
    'min_child_weight':1.24,
    'objective':'reg:squarederror',
    'reg_alpha':0.28,
    'reg_lambda':0.004,
    'seed':42,   
    }
    mlflow.log_params(params)
        
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=20,
        evals=[(valid,'validation')],
        early_stopping_rounds=5,
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val,y_pred)
    mlflow.log_metric('rmse',rmse)
    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')



    

ERROR! Session/line number was not unique in database. History logging moved to new session 660


2025/05/24 15:33:13 WARNING mlflow.xgboost: Failed to log dataset information to MLflow Tracking. Reason: Unable to allocate 5.77 GiB for an array with shape (3009173, 515) and data type float32


[0]	validation-rmse:9.31803
[1]	validation-rmse:8.80850
[2]	validation-rmse:8.45409
[3]	validation-rmse:8.17959
[4]	validation-rmse:7.99497
[5]	validation-rmse:7.87127
[6]	validation-rmse:7.78404
[7]	validation-rmse:7.62483
[8]	validation-rmse:7.50688
[9]	validation-rmse:7.45840
[10]	validation-rmse:7.41475
[11]	validation-rmse:7.38010
[12]	validation-rmse:7.34836
[13]	validation-rmse:7.31658
[14]	validation-rmse:7.26960
[15]	validation-rmse:7.19015
[16]	validation-rmse:7.17313
[17]	validation-rmse:7.15488
[18]	validation-rmse:7.10211
[19]	validation-rmse:7.08011


2025/05/24 15:33:22 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/05/24 15:33:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "D:\cursos\mlops_zoomcamp\homework2\env_hw2\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [15:33:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/05/24 15:33:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
D:\cursos\mlops_zoomcamp\homework2\env_hw2\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [15:33:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c

In [33]:

mlflow.xgboost.autolog(disable=True)

## Saving preprocessor

In [34]:
with mlflow.start_run():
    params={
    'learning_rate':0.205,
    'max_depth':18,
    'min_child_weight':1.24,
    'objective':'reg:squarederror',
    'reg_alpha':0.28,
    'reg_lambda':0.004,
    'seed':42,   
    }
    mlflow.log_params(params)
        
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=20,
        evals=[(valid,'validation')],
        early_stopping_rounds=5,
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val,y_pred)
    mlflow.log_metric('rmse',rmse)

    with open('models/preprocessor.b','wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')



    
    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')



    

[0]	validation-rmse:9.31803
[1]	validation-rmse:8.80850
[2]	validation-rmse:8.45409
[3]	validation-rmse:8.17959
[4]	validation-rmse:7.99497
[5]	validation-rmse:7.87127
[6]	validation-rmse:7.78404
[7]	validation-rmse:7.62483
[8]	validation-rmse:7.50688
[9]	validation-rmse:7.45840
[10]	validation-rmse:7.41475
[11]	validation-rmse:7.38010
[12]	validation-rmse:7.34836
[13]	validation-rmse:7.31658
[14]	validation-rmse:7.26960
[15]	validation-rmse:7.19015
[16]	validation-rmse:7.17313
[17]	validation-rmse:7.15488
[18]	validation-rmse:7.10211
[19]	validation-rmse:7.08011


D:\cursos\mlops_zoomcamp\homework2\env_hw2\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [15:39:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/24 15:39:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
